<a href="https://colab.research.google.com/github/kylemcq13/Projects/blob/master/16_8_Feature_Engineering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment

Say, we want to understand the relationship between the expenditures of the governments and the students' overall success in the math and reading.

1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.
2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?
3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?
4. What is the correlation between the overall score variable and the 1st principal component? 
5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [0]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

edu_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

In [0]:
#interpolate the data as before

columns = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE", 
           "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE", 
           "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
           "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G", 
           "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G", 
           "GRADES_9_12_G", "GRADES_ALL_G", "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", 
           "AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]

edu5_df = edu_df.copy()
states = edu5_df["STATE"].unique()

for col in columns:
    for state in states:
        edu5_df.loc[edu5_df["STATE"] == state, col] = edu5_df.loc[edu5_df["STATE"] == state, col].fillna(edu5_df[edu5_df["STATE"] == state][col].interpolate())
        
edu5_df.dropna(inplace=True)
edu5_df

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
54,1993_ARIZONA,ARIZONA,1993,676297.0,3427976.0,318465.0,1415407.0,1694104.0,3623946.0,1578889.0,...,57701.0,53500.0,37614.0,461398.0,182737.0,647299.0,215.834434,265.993417,206.291984,262.448330
55,1993_ARKANSAS,ARKANSAS,1993,311432.0,1346909.0,128196.0,771079.0,447634.0,1376067.0,782791.0,...,34255.0,36471.0,27169.0,280280.0,125801.0,407329.0,211.616130,257.647254,208.685273,266.423396
57,1993_COLORADO,COLORADO,1993,539538.0,3058326.0,147793.0,1242337.0,1668196.0,3028305.0,1537714.0,...,50807.0,47665.0,34770.0,403871.0,164260.0,575380.0,222.219036,273.200878,215.523979,266.856804
58,1993_CONNECTICUT,CONNECTICUT,1993,471918.0,4064158.0,149204.0,1433539.0,2481415.0,4079943.0,2302852.0,...,39093.0,34662.0,27979.0,310022.0,127655.0,443893.0,228.105503,275.202320,224.712775,264.457874
59,1993_DELAWARE,DELAWARE,1993,104355.0,683954.0,44497.0,443250.0,196207.0,694534.0,394680.0,...,8285.0,8265.0,5882.0,67899.0,28930.0,97394.0,217.181255,263.834857,207.792430,262.255809
60,1993_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1993,80937.0,721151.0,73882.0,0.0,647269.0,719440.0,339266.0,...,5870.0,4935.0,3303.0,46917.0,17854.0,69987.0,191.234082,234.401343,179.324956,265.717190
61,1993_FLORIDA,FLORIDA,1993,1980569.0,11370156.0,947326.0,5520893.0,4901937.0,11425183.0,5333136.0,...,168509.0,149054.0,100646.0,1313261.0,525569.0,1873623.0,214.208209,260.841485,205.486357,260.395898
106,1994_ARIZONA,ARIZONA,1994,708482.0,3566586.0,348847.0,1478372.0,1739367.0,3803379.0,1652311.0,...,60032.0,55334.0,40119.0,475475.0,194196.0,673211.0,216.414936,266.620556,206.371252,262.726766
107,1994_ARKANSAS,ARKANSAS,1994,312695.0,1393287.0,132108.0,810396.0,450783.0,1445984.0,816270.0,...,34488.0,36437.0,27161.0,279964.0,127437.0,409053.0,213.026232,258.982418,208.736087,268.227127
109,1994_COLORADO,COLORADO,1994,549570.0,3061865.0,165539.0,1269532.0,1626794.0,3113755.0,1574164.0,...,51877.0,49332.0,35464.0,410452.0,170200.0,590505.0,223.414643,274.003324,217.567067,267.293921


# 1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [0]:
#using a weight factor of 0.5 for each grade, weight the scores based on student population of each grade

edu5_df['weighted_scores'] = (edu5_df['GRADES_4_G'] * ((edu5_df['AVG_MATH_4_SCORE'] + edu5_df['AVG_READING_4_SCORE']) * 0.5) + edu5_df['GRADES_8_G'] * ((edu5_df['AVG_MATH_8_SCORE'] + edu5_df['AVG_READING_8_SCORE']) * 0.5)) / (edu5_df['GRADES_4_G'] + edu5_df['GRADES_8_G'])
edu5_df

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,weighted_scores
54,1993_ARIZONA,ARIZONA,1993,676297.0,3427976.0,318465.0,1415407.0,1694104.0,3623946.0,1578889.0,...,53500.0,37614.0,461398.0,182737.0,647299.0,215.834434,265.993417,206.291984,262.448330,236.637935
55,1993_ARKANSAS,ARKANSAS,1993,311432.0,1346909.0,128196.0,771079.0,447634.0,1376067.0,782791.0,...,36471.0,27169.0,280280.0,125801.0,407329.0,211.616130,257.647254,208.685273,266.423396,236.905842
57,1993_COLORADO,COLORADO,1993,539538.0,3058326.0,147793.0,1242337.0,1668196.0,3028305.0,1537714.0,...,47665.0,34770.0,403871.0,164260.0,575380.0,222.219036,273.200878,215.523979,266.856804,243.634022
58,1993_CONNECTICUT,CONNECTICUT,1993,471918.0,4064158.0,149204.0,1433539.0,2481415.0,4079943.0,2302852.0,...,34662.0,27979.0,310022.0,127655.0,443893.0,228.105503,275.202320,224.712775,264.457874,246.815311
59,1993_DELAWARE,DELAWARE,1993,104355.0,683954.0,44497.0,443250.0,196207.0,694534.0,394680.0,...,8265.0,5882.0,67899.0,28930.0,97394.0,217.181255,263.834857,207.792430,262.255809,237.735539
60,1993_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1993,80937.0,721151.0,73882.0,0.0,647269.0,719440.0,339266.0,...,4935.0,3303.0,46917.0,17854.0,69987.0,191.234082,234.401343,179.324956,265.717190,214.866567
61,1993_FLORIDA,FLORIDA,1993,1980569.0,11370156.0,947326.0,5520893.0,4901937.0,11425183.0,5333136.0,...,149054.0,100646.0,1313261.0,525569.0,1873623.0,214.208209,260.841485,205.486357,260.395898,233.677772
106,1994_ARIZONA,ARIZONA,1994,708482.0,3566586.0,348847.0,1478372.0,1739367.0,3803379.0,1652311.0,...,55334.0,40119.0,475475.0,194196.0,673211.0,216.414936,266.620556,206.371252,262.726766,236.948517
107,1994_ARKANSAS,ARKANSAS,1994,312695.0,1393287.0,132108.0,810396.0,450783.0,1445984.0,816270.0,...,36437.0,27161.0,279964.0,127437.0,409053.0,213.026232,258.982418,208.736087,268.227127,237.967381
109,1994_COLORADO,COLORADO,1994,549570.0,3061865.0,165539.0,1269532.0,1626794.0,3113755.0,1574164.0,...,49332.0,35464.0,410452.0,170200.0,590505.0,223.414643,274.003324,217.567067,267.293921,244.939105


# 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [0]:
#created a correlation table of weighted scores and expenditures

edu5_df[['weighted_scores','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE']].corr()

,weighted_scores,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE
weighted_scores,1.000000,0.093020,0.105787,0.098643,0.027635
TOTAL_EXPENDITURE,0.093020,1.000000,0.990829,0.993175,0.946483
INSTRUCTION_EXPENDITURE,0.105787,0.990829,1.000000,0.975867,0.907235
SUPPORT_SERVICES_EXPENDITURE,0.098643,0.993175,0.975867,1.000000,0.953789
OTHER_EXPENDITURE,0.027635,0.946483,0.907235,0.953789,1.000000


It seems as if the instruction expenditure has the most correlation with the weighted scores.

# 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [0]:
#calculate the PCA of the 4 expenditure types

pca_exp = edu5_df[['INSTRUCTION_EXPENDITURE', 'TOTAL_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE']]

pca_exp = StandardScaler().fit_transform(pca_exp)

sklearn_pca = PCA(n_components=1)
edu5_df['pca_col'] = sklearn_pca.fit_transform(pca_exp)

print(sklearn_pca.explained_variance_ratio_)

[0.97107935]


According to the calculation, over 97% of the variance is explained by the 1st component.

# 4. What is the correlation between the overall score variable and the 1st principal component?

In [0]:
#produce a correlation table

edu5_df[['weighted_scores', 'pca_col', 'INSTRUCTION_EXPENDITURE', 'TOTAL_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE']].corr()

,weighted_scores,pca_col,INSTRUCTION_EXPENDITURE,TOTAL_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE
weighted_scores,1.000000,0.082805,0.105787,0.093020,0.098643,0.027635
pca_col,0.082805,1.000000,0.983098,0.997324,0.995350,0.965638
INSTRUCTION_EXPENDITURE,0.105787,0.983098,1.000000,0.990829,0.975867,0.907235
TOTAL_EXPENDITURE,0.093020,0.997324,0.990829,1.000000,0.993175,0.946483
SUPPORT_SERVICES_EXPENDITURE,0.098643,0.995350,0.975867,0.993175,1.000000,0.953789
OTHER_EXPENDITURE,0.027635,0.965638,0.907235,0.946483,0.953789,1.000000


As can be seen from the table above, the correlation between the first component and weighted score variable is 0.0828 or roughtly 8.3%.

# 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

According to our table, the expenditures are more correlated with our weighted scores than the principal component. Of the expenditures, instruction expenditure is the most correlated so it would make sense to use this. Also, according to the literature, PCA works best on variables that are correlated to 0.8 or lower. In our case, the expenditures correlations all exceed this value which could lead to problems. 